<a href="https://colab.research.google.com/github/saikishorereddy533/Emotion-Recognition-Through-Speech/blob/main/Emotion_Recognition_Through_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
Root = "/content/drive/MyDrive/speech-emotion-recognition-ravdess-data"
os.chdir(Root)

In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
x_train

array([[-5.95531372e+02,  6.62860336e+01,  1.57717609e+01, ...,
         2.94203764e-05,  7.37478376e-06,  3.24825965e-06],
       [-6.75629639e+02,  7.35061798e+01,  1.95937097e+00, ...,
         4.37001654e-06,  3.66387007e-06,  1.56673582e-06],
       [-6.50842468e+02,  5.31168976e+01, -3.24820185e+00, ...,
         5.32868944e-05,  4.49146064e-05,  2.97620154e-05],
       ...,
       [-7.09466431e+02,  2.33863258e+01, -8.97838879e+00, ...,
         2.92042409e-06,  2.66227448e-06,  4.61045556e-06],
       [-7.20267334e+02,  4.92638626e+01,  1.90532093e+01, ...,
         4.06075560e-06,  1.23242228e-06,  5.81119480e-07],
       [-5.23919434e+02,  3.89732323e+01, -1.22774239e+01, ...,
         1.16796538e-04,  6.61305821e-05,  5.32570921e-05]])

In [ ]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [ ]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
y_pred

array(['happy', 'fearful', 'happy', 'calm', 'calm', 'fearful', 'fearful',
       'fearful', 'fearful', 'calm', 'calm', 'happy', 'calm', 'happy',
       'fearful', 'fearful', 'calm', 'happy', 'fearful', 'fearful',
       'fearful', 'fearful', 'calm', 'happy', 'happy', 'disgust', 'happy',
       'disgust', 'happy', 'fearful', 'fearful', 'fearful', 'happy',
       'calm', 'calm', 'fearful', 'fearful', 'happy', 'happy', 'fearful',
       'fearful', 'fearful', 'fearful', 'calm', 'calm', 'fearful',
       'fearful', 'happy', 'fearful', 'happy', 'disgust', 'happy',
       'disgust', 'calm', 'happy', 'disgust', 'fearful', 'calm',
       'disgust', 'calm', 'fearful', 'calm', 'fearful', 'calm', 'happy',
       'fearful', 'fearful', 'calm', 'calm', 'calm', 'calm', 'disgust',
       'disgust', 'fearful', 'disgust', 'fearful', 'disgust', 'calm',
       'fearful', 'calm', 'fearful', 'fearful', 'calm', 'happy',
       'fearful', 'disgust', 'fearful', 'happy', 'fearful', 'fearful',
       'fearful', '

In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 67.71%


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
f1_score(y_test, y_pred,average=None)

array([0.85454545, 0.57142857, 0.688     , 0.51162791])

In [ ]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,disgust,happy
1,happy,fearful
2,happy,happy
3,calm,calm
4,disgust,calm
5,fearful,fearful
6,happy,fearful
7,calm,fearful
8,disgust,fearful
9,calm,calm
